This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
params = dict(limit=1, api_key=API_KEY)
request = requests.get(url, params=params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response = request.json()
response.keys()

dict_keys(['dataset'])

In [5]:
response['dataset']

{'id': 10095370,
 'dataset_code': 'AFX_X',
 'database_code': 'FSE',
 'name': 'Carl Zeiss Meditec (AFX_X)',
 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
 'refreshed_at': '2020-12-01T14:48:09.907Z',
 'newest_available_date': '2020-12-01',
 'oldest_available_date': '2000-06-07',
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'frequency': 'daily',
 'type': 'Time Series',
 'premium': False,
 'limit': 1,
 'transform': None,
 'column_index': None,
 'start_date': '2000-06-07',
 'end_date': '2020-12-01',
 'data': [['2020-12-01',
   112.2,
   112.2,
   111.5,
   112.0,
   None,
   51.0,
   5703.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None,
 'database_id': 6129}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### requesting the stock data and save it as a dictionary

In [6]:
def get_stocks_data(url, start, end , api_key = API_KEY):
    params = {'start_date': start, 'end_date': end, 'api_key':API_KEY}
    request = requests.get(url, params=params)
    if not request:
        print(f'error:{request.status_code}')
        return
    else:
        return request.json()['dataset']
    

In [7]:
stocks_data = get_stocks_data(url, '2017-01-01', '2017-12-31')

### save data as a namedtuple

In [8]:
data = collections.namedtuple('data', ['Date','Open','High','Low','Close','Volume'])
tuple_data = [data(day_data[0], day_data[1], day_data[2], day_data[3], day_data[4], day_data[6]) 
              for day_datain stocks_data['data']]

### highest and lowest opening prices 

In [9]:
open_data = [i.Open for i in tuple_data if i.Open!=None]
print(f'maximum opening price was {max(open_data)}')
print(f'minimum opening price was {min(open_data)}')

maximum opening price was 53.11
minimum opening price was 34.0


### largest change in any one day (based on High and Low price)

In [10]:
change = [i.High-i.Low for i in tuple_data if i.High and i.Low !=None]
print(f'maximum change in one day was {round(max(change),2)}')

maximum change in one day was 2.81


### largest change between any two days (based on Closing Price)

In [11]:
close=[i.Close for i in tuple_data if i.Close!=None]
def find_max_close_between_days(close):
    max_diff = abs(close[1]-close[0])
    for i in range(len(close)-2):
        diff = abs(close[i+1]-close[i])
        if diff > max_diff:
            max_diff = diff
    return round(max_diff,2)
print(f'largest change between any two successive days was: {find_max_close_between_days(close)}')

largest change between any two successive days was: 2.56


### average daily trading volume during this year

In [12]:
volume=[i.Volume for i in tuple_data if i.Volume!=None]
print(f'average daily trading volume during this year was: {round(sum(volume)/len(volume),2)}')

average daily trading volume during this year was: 89124.34


### median trading volume during this year

In [15]:
def find_median(volume):
    volume = sorted(volume)
    if len(volume)%2!=0:
        idx = int((len(volume)+1)/2)
        return volume[idx]
    else:
        idx = int(len(volume)/2)
        return (volume[idx]+volume[idx+1])/2
print(f'median trading volume during this year was: {find_median(volume)}')

median trading volume during this year was: 76600.0
